![](aws/RDS_Images/1.png)
**select create database**

![](aws/RDS_Images/2.png)
**after choosing mysql click on the free tier**

![](aws/RDS_Images/3.png)

![](aws/RDS_Images/4.png)

![](aws/RDS_Images/5.png)

![](aws/RDS_Images/6.png)

![](aws/RDS_Images/7.png)

![](aws/RDS_Images/8.png)

![](aws/RDS_Images/9.png)

![](aws/RDS_Images/10.png)

![](aws/RDS_Images/11.png)

![](aws/RDS_Images/12.png)

![](aws/RDS_Images/13.png)

![](aws/RDS_Images/14.png)

![](aws/RDS_Images/15.png)

In [33]:
!pip install pymysql

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [111]:
import pymysql.cursors
import pandas as pd

import csv

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [53]:
connection = pymysql.connect(
    host = 'myawssql.cytf4bcdtf95.us-east-1.rds.amazonaws.com',
    port = 3306,
    user = 'nikawssql',
    password = 'nikawssql',
    database='mydb'
    )
cursor = connection.cursor()

Let's see how many databases are here

In [54]:
cursor.execute("""show databases""")

pd.read_sql("show databases", con = connection)

6

,Database
0,information_schema
1,innodb
2,mydb
3,mysql
4,performance_schema
5,sys


we can see that our 'mydb' database is there

And we currently have no tables in our database

Let's create tables, we will be using Transaction data
We will create 7 tables

![](https://web.archive.org/web/20161019192412im_/http://lisp.vse.cz/pkdd99/data.gif)

[Guide to the Financial Data Set](https://web.archive.org/web/20161019192412/http://lisp.vse.cz/pkdd99/berka.htm)
    
[1999 Czech Financial Dataset - Real Anonymized Transactions](https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions)

In [20]:
cursor.execute("""CREATE TABLE demograph
(
   district_id int primary key,
   name text,
   place text,
   a0 INT,
   a1 float,
   a2 float,
   a3 float,
   a4 float,
   a5 float,
   a6 float,
   a7 float,
   a8 float,
   a9 float,
   a10 float,
   a11 float,
   a12 float
)""")
               
cursor.execute("""CREATE TABLE account
(
    account_id int primary key,
    district_id int,
    frequency text,
    dated datetime,
    Constraint fk_demograph_account Foreign Key (district_id) References demograph(district_id)
)""")               
               
cursor.execute("""CREATE TABLE loan(
    loan_id int,
    account_id int, 
    date timestamp, 
    amount decimal(20, 3),
    duration int,
    payments decimal(20, 3),
    status text,
    Constraint fk_account_loan Foreign Key (account_id) References account(account_id)
)""")              
               
cursor.execute("""CREATE TABLE permanent_order
(
    order_id int,
    account_id int, 
    bank_to text, 
    account_to int,
    amount float,
    k_symbol text,
    Constraint fk_account_order Foreign Key (account_id) References account(account_id)
)""")               
               
cursor.execute("""CREATE TABLE client
(
    client_id int primary key,
    birth_number int,
    district_id int,
    Constraint fk_demograph_client Foreign Key (district_id) References demograph(district_id)
)""")             
               
cursor.execute("""CREATE TABLE disposition
(
    disp_id int primary key,
    client_id int,
    account_id int,
    type text,
    Constraint fk_account_disposition Foreign Key (account_id) References account(account_id),
    Constraint fk_client_disposition Foreign Key (client_id) References client(client_id)
)""")                
               
cursor.execute("""CREATE TABLE credit_card
(
    card_id int,
    disp_id int,
    type text,
    issued text,
    Constraint fk_disposition_credit_card Foreign Key (disp_id) References disposition(disp_id)
)""")               

0

Loading data into tables from csv files

In [ ]:
#### demograph
with open('Anti_money_Laundering/district.csv', newline = '') as csvfile:
    next(csvfile)
    x = csv.reader(csvfile, delimiter = ';')
    
    for row in x:
        cursor.execute("""INSERT INTO demograph (district_id, name, place, a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", row)
    #cursor.close()
print("Imported district")

#### account
with open('Anti_money_Laundering/account.csv', newline='') as csvfile:
    next(csvfile)
    x = csv.reader(csvfile, delimiter=';')
    
    for row in x:
        cursor.execute("""INSERT INTO account (account_id, district_id, frequency, dated)
        VALUES(%s, %s, %s, %s)""", row)
    #cursor.close()
connection.commit()
print("Imported account")

#### loan
with open('Anti_money_Laundering/loan.csv', newline='') as csvfile:
    next(csvfile)
    x = csv.reader(csvfile, delimiter=';')
    
    for row in x:
        cursor.execute("""INSERT INTO loan (loan_id, account_id, date, amount, duration, payments, status)
        VALUES(%s, %s, %s, %s, %s, %s, %s)""", row)
    #cursor.close()
connection.commit()
print("Imported loan")


#### permanent_order
with open('Anti_money_Laundering/order.csv', newline='') as csvfile:
    next(csvfile)
    x = csv.reader(csvfile, delimiter=';')
    
    for row in x:
        cursor.execute("""INSERT INTO permanent_order (order_id, account_id, bank_to, account_to, amount, k_symbol)
        VALUES(%s, %s, %s, %s, %s, %s)""", row)
    #cursor.close()
connection.commit()
print("Imported permanent_order")

#### client
with open('Anti_money_Laundering/client.csv', newline='') as csvfile:
    next(csvfile)
    x = csv.reader(csvfile, delimiter=';')
    
    for row in x:
        cursor.execute("""INSERT INTO client (client_id, birth_number, district_id)
        VALUES(%s, %s, %s)""", row)
    #cursor.close()
connection.commit()
print("Imported client")


#### disposition
with open('Anti_money_Laundering/disp.csv', newline='') as csvfile:
    next(csvfile)
    x = csv.reader(csvfile, delimiter=';')
    
    for row in x:
        cursor.execute("""INSERT INTO disposition (disp_id, client_id, account_id, type)
        VALUES(%s, %s, %s, %s)""", row)
    #cursor.close()
connection.commit()
print("Imported disposition")


#### credit_card
with open('Anti_money_Laundering/card.csv', newline='') as csvfile:
    next(csvfile)
    x = csv.reader(csvfile, delimiter=';')
    
    for row in x:
        cursor.execute("""INSERT INTO credit_card (card_id, disp_id, type, issued)
        VALUES(%s, %s, %s, %s)""", row)
    #cursor.close()
connection.commit()
print("Imported credit_card")

In [78]:
pd.read_sql("""SELECT d.name, d.place, a.dated, a.frequency from demograph d JOIN account a
                ON d.district_id = a.district_id group by place""", con = connection)

,name,place,dated,frequency
0,Benesov,central Bohemia,1996-06-12,POPLATEK MESICNE
1,Havlickuv Brod,east Bohemia,1996-11-24,POPLATEK MESICNE
2,Ceska Lipa,north Bohemia,1993-05-10,POPLATEK MESICNE
3,Bruntal,north Moravia,1993-09-17,POPLATEK TYDNE
4,Hl.m. Praha,Prague,1993-02-26,POPLATEK MESICNE
5,Ceske Budejovice,south Bohemia,1997-04-25,POPLATEK MESICNE
6,Blansko,south Moravia,1995-03-10,POPLATEK MESICNE
7,Domazlice,west Bohemia,1993-08-26,POPLATEK MESICNE


In [112]:
cursor.execute("""show tables""")
pd.read_sql("show tables", con = connection)

7

,Tables_in_mydb
0,account
1,client
2,credit_card
3,demograph
4,disposition
5,loan
6,permanent_order
